# Install and import libraries

In [1]:
!pip install einops
!pip install datasets
!pip install torch-fidelity
!pip install torchmetrics
!pip3 install image-similarity-measures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 489 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 15.1 MB/s 
     |████████████████████████████████| 182 kB 70.7 MB/s 
     |████████████████████████████████| 115 kB 56.3 MB/s 
     |████████████████████████████████| 212 kB 54.5 MB/s 
     |████████████████████████████████| 127 kB 56.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 14.3 MB/s 
Looking in indexes: https://pypi.org/simp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import numpy
from numpy import cov, trace, iscomplexobj
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import sys

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Paths (Checkpoint)

In [4]:
# FashionCNN_checkpoint = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/fashion_mnist_CNNClassifier_91Acc.pt"
DiffusionModel_checkpoint = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/mnist/checkpoint_T400_linear/mnist/Unet.ckpt"

Stats_path = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/mnist/checkpoint_T400_linear/mnist/stats_mnist_T400_linear.csv"

# Load FashionMNIST Dataset to get Real Images

In [5]:
train_set = torchvision.datasets.FashionMNIST("./data", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=100, shuffle=False)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



# Generate Images from trained Diffusion Model

In [6]:
!git clone https://github.com/GangaMegha/Diffusion-Models.git 

Cloning into 'Diffusion-Models'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 191 (delta 105), reused 136 (delta 56), pack-reused 0
Receiving objects: 100% (191/191), 49.31 KiB | 9.86 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [7]:
%cd /content/Diffusion-Models/src
!ls

/content/Diffusion-Models/src
config.py	diffusion  main.py  train
data_loader.py	logger.py  models   visualize


#### Load Model

In [9]:
from models.unet import Unet as Model
from config import LEVEL, TRAIN, MODEL, MODEL_TYPE, DATASET_NAME, CONFIG, CHECKPOINT_PATH, RESULT_PATH, LOG_PATH

from diffusion.reverse_data_generate import sample
from diffusion.var_schedule import alpha_beta

from data_loader import reverse_transform

cfg = CONFIG[MODEL_TYPE.value]["mnist"]
Noise_model = Model(cfg)
Noise_model_checkpoint = torch.load(DiffusionModel_checkpoint)
Noise_model.load_state_dict(Noise_model_checkpoint['state_dict'])
Noise_model.to(device)
Noise_model.eval()

variance_dict = alpha_beta(T=400, schedule=cfg.get("var_schedule"))

#### Generate Images

In [10]:
fake_images = []
for i in range(10):
  fake_images_list = sample(Noise_model, variance_dict, cfg, sample_cnt=100, every50=False)
  fake_images.append(reverse_transform()(torch.cat(fake_images_list, dim=0)))

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

# Compute True FID and IS using Inception Network

In [11]:
from train.metrics import FID, IS

In [12]:
FID_ = FID().to(device)
IS_ = IS().to(device)

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth


  0%|          | 0.00/91.2M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [16]:
it = iter(train_loader)
FID_list, IS_list = [], []

for i in range(10):
  batch = next(it)[0]
  FID_list.append(FID_(fake_images[i].to(device), (batch*255).to(dtype=torch.uint8).to(device), cfg["grayscale"]).item())
  IS_list.append(IS_(fake_images[i].to(device), cfg["grayscale"]).item())

print("FID_list : ", FID_list)
print("IS_list : ", IS_list)

print("FID Mean : ", np.mean(np.array(FID_list)))
print("FID Std : ", np.std(np.array(FID_list)))

print("IS Mean : ", np.mean(np.array(IS_list)))
print("IS Std : ", np.std(np.array(IS_list)))

FID_list :  [279.0436706542969, 272.626953125, 271.8067626953125, 268.3907470703125, 269.7808837890625, 266.00604248046875, 267.2637634277344, 272.6766662597656, 271.94482421875, 267.0625915527344]
IS_list :  [1.5965325832366943, 1.6984306573867798, 1.6082178354263306, 1.6256039142608643, 1.6328963041305542, 1.5957136154174805, 1.6622387170791626, 1.635170817375183, 1.580204963684082, 1.5579477548599243]
FID Mean :  270.66029052734376
FID Std :  3.650274829426012
IS Mean :  1.6192957162857056
IS Std :  0.0388225229321831


# Save Computed stats

In [17]:
# load csv module
import csv

# define a dictionary with key value pairs
diction = {
    "Metric" : "Value",
    "FID Mean" : np.mean(np.array(FID_list)),
    "FID Std" : np.std(np.array(FID_list)),
    "IS Mean" : np.mean(np.array(IS_list)),
    "IS Std" : np.std(np.array(IS_list))
    }

# open file for writing, "w" is writing
w = csv.writer(open(Stats_path, "w"))

# loop over dictionary keys and values
for key, val in diction.items():

    # write every key and value to file
    w.writerow([key, val])
